# GO annotation analyses

1. Find GO annotations in GFF file for each genome
2. Run through the files and come up with a list of unique GO IDs
3. Create a table with presence/absence for all genomes

### 1. Find GO annotations in GFF file for each genome, save each list to an individual file

In [1]:
import csv

def get_modeling_accessions():
    with open("results/2_JGIgold_KEGG_anayses_out/05_genome_feature_matrix.tsv", "r", newline="") as rf:
        next(rf)
        reader = csv.reader(rf, delimiter="\t")
        accessions = [row[0] for row in reader]
    return accessions

In [2]:
import os

def find_go_annotations(gff_file):
    go_terms = set([])
    with open(gff_file, "r") as rf:
        for line in rf:
            if not line.startswith("#"):
                fields = line.split("\t")[-1].split(";")
                for field in fields:
                    if field.startswith("Ontology_term"):
                        field_data = field[field.index("=")+1:]
                        go_terms.update([term[3:] for term in field_data.split(",")])

    genome = gff_file.split("/")[-2]
    outfile = "/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/GO_lists/" + genome + ".txt"
    with open(outfile, "w") as wf:
        for term in go_terms:
            wf.write(term + "\n")

In [8]:
test_id = "GCF_030643905.1"
test_path = "/n/scratch/users/b/byc014/github/bac_genome_constraint/data/ncbi/assemblies/GCF_030643905.1/genomic.gff"

find_go_annotations(test_path)

In [5]:
import glob
import os

os.chdir("/n/scratch/users/a/aip485/bac_genome_constraint/")
modeling_accessions = get_modeling_accessions()

with open("/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/missing_genomes.txt", "w") as wf:
    for acc in modeling_accessions:
        gff_file = f"/n/scratch/users/b/byc014/github/bac_genome_constraint/data/ncbi/assemblies/{acc}/genomic.gff"
        if os.path.exists(gff_file):
            find_go_annotations(gff_file)
        else:
            print("Missing", acc)
            wf.write(acc + "\n")

### 2. Run through the files and come up with a list of unique GO IDs

In [6]:
import os

unique_terms = set([])
to_skip = []
for go_list_file in glob.iglob("/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/GO_lists/*.txt"):
    with open(go_list_file, "r") as rf:
        genome_terms = [int(line.strip()) for line in rf]
        if len(genome_terms) == 0:
            to_skip.append(os.path.basename(go_list_file))
        unique_terms.update(genome_terms)

with open("/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/unique_terms.txt", "w") as wf:
    for term in sorted(unique_terms):
        wf.write(f"{term:07d}\n")

with open("/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/missing_terms.txt", "w") as wf:
    for genome in to_skip:
        wf.write(genome + "\n")

#### 2.1 Remove files without any GO terms

In [7]:
import os

with open("/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/missing_terms.txt", "r") as rf:
    missing_terms = [line.strip() for line in rf]

for basename in missing_terms:
    fn = "/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/GO_lists/" + basename
    os.remove(fn)

### 3. Create a table with presence/absence for all genomes

In [8]:
with open("/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/unique_terms.txt", "r") as rf:
    unique_terms = [line.strip() for line in rf]

genome_match_string = "/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/GO_lists/*.txt"
genome_files = [fn for fn in glob.iglob(genome_match_string) ]

with open("/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/presence_absence_table.txt", "w") as wf:

    # Write header
    wf.write("Genome\t" + "\t".join(unique_terms) + "\n")

    for fn in glob.iglob("/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/GO_lists/*.txt"):

        basename = os.path.basename(fn)
        
        with open(fn, "r") as rf:
            genome_terms = [line.strip() for line in rf]
        presence_absence = ["1" if term in genome_terms else "0" for term in unique_terms]
        wf.write(basename[:-4] + "\t" + "\t".join(presence_absence) + "\n")


### Create list of ubiquitous terms (present in >95% of genomes)

In [9]:
import glob
import os

with open("/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/unique_terms.txt", "r") as rf:
    unique_terms = [line.strip() for line in rf]

term_counts = {term: 0 for term in unique_terms}

filenames = glob.glob("/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/GO_lists/*.txt")
for fn in filenames:
    basename = os.path.basename(fn)
    with open(fn, "r") as rf:
        for line in rf:
            term = line.strip()
            term_counts[term] += 1

abundance_cutoff = len(filenames) * 0.95
print(len(filenames), "files, cutoff is", abundance_cutoff)

ubiquitous_terms_count = 0
with open("/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/ubiquitous_terms.txt", "w") as wf:
    for term in unique_terms:
        if term_counts[term] >= abundance_cutoff:
            wf.write(term + "\n")
            ubiquitous_terms_count += 1

print(ubiquitous_terms_count, "ubiquitous terms")

3088 files, cutoff is 2933.6
334 ubiquitous terms


### Generate per-genome counts for all terms

In [10]:
def find_go_annotations_with_counts(gff_file):
    go_terms = {}
    with open(gff_file, "r") as rf:
        for line in rf:
            if not line.startswith("#"):
                fields = line.split("\t")[-1].split(";")
                for field in fields:
                    if field.startswith("Ontology_term"):
                        field_data = field[field.index("=")+1:]
                        for full_term in field_data.split(","):
                            term = full_term[3:]
                            go_terms[term] = go_terms.get(term, 0) + 1

    genome = gff_file.split("/")[-2]
    outfile = "/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/GO_lists_with_counts/" + genome + ".txt"
    with open(outfile, "w") as wf:
        for term, count in go_terms.items():
            wf.write(f"{term}\t{count}\n")

In [11]:
import os

modeling_accessions = get_modeling_accessions()

for acc in modeling_accessions:
    gff_file = f"/n/scratch/users/b/byc014/github/bac_genome_constraint/data/ncbi/assemblies/{acc}/genomic.gff"
    find_go_annotations_with_counts(gff_file)

In [12]:
import os

with open("/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/missing_terms.txt", "r") as rf:
    missing_terms = [line.strip() for line in rf]

for basename in missing_terms:
    fn = "/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/GO_lists_with_counts/" + basename
    os.remove(fn)

### Generate table of ubiquitous term counts for all genomes

In [13]:
with open("/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/ubiquitous_terms.txt", "r") as rf:
    ubiquitous_terms = [line.strip() for line in rf]
term_index_map = {term: i for i, term in enumerate(ubiquitous_terms)}
ubiquitous_term_count = len(ubiquitous_terms)

with open("/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/ubiquitous_counts_table.txt", "w") as wf:
    wf.write("Genome\t" + "\t".join(ubiquitous_terms) + "\n")
    for fn in glob.iglob("/n/scratch/users/a/aip485/bac_genome_constraint/results/3_GO_analyses/GO_lists_with_counts/*.txt"):
        basename = os.path.basename(fn)

        counts_list = ['0' for i in range(ubiquitous_term_count)]
        with open(fn, "r") as rf:
            for line in rf:
                fields = line.strip().split("\t")
                if fields[0] in term_index_map:
                    counts_list[term_index_map[fields[0]]] = fields[1]
        
        wf.write(basename[:-4] + "\t" + "\t".join(counts_list) + "\n")


### Normalize term counts by number of protein coding genes

In [ ]:
import csv
import os
import pandas as pd

genome_sizes = pd.read_csv(
    "results/2_JGIgold_KEGG_anayses_out/05_genome_feature_matrix.tsv", 
    sep="\t", 
    usecols=["accession", "genes_proteinCoding"],
    index_col="accession",
)

with open("results/3_GO_analyses/ubiquitous_counts_table.txt", "r", newline="") as rf, \
     open("results/3_GO_analyses/ubiquitous_counts_table_normalized.txt", "w") as wf:
    wf.write(next(rf))
    reader = csv.reader(rf, delimiter="\t")
    for row in reader:
        accession = row[0]
        if accession in genome_sizes.index:
            ngenes = int(genome_sizes.at[accession, "genes_proteinCoding"])
            wf.write(accession + "\t" + "\t".join([str(int(count) / ngenes) for count in row[1:]]) + "\n")